In [47]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc

from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import Callback
from keras.models import Sequential
from keras.optimizers import Adam

In [48]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

In [49]:
test_df = pd.read_csv('test_data_code2.csv')
test_df.head()

,Unnamed: 0,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,184903890,9,107,0.0,2017-11-10 04:00:00,1,5744,NaN,3,4,10,4,34,113,3
1,184903891,9,466,1.0,2017-11-10 04:00:00,1,119901,NaN,3,4,10,4,403,2302,27
2,184903892,21,128,2.0,2017-11-10 04:00:00,1,72287,NaN,19,4,10,4,229,701,121
3,184903893,15,111,3.0,2017-11-10 04:00:00,1,78477,NaN,13,4,10,4,239,827,261
4,184903894,12,328,4.0,2017-11-10 04:00:00,1,123080,NaN,13,4,10,4,60,205,38


In [50]:
sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']

In [51]:
test_df.drop(['Unnamed: 0','click_time','ip','is_attributed'],1,inplace=True)
# test_df = get_keras_data(test_df)

In [52]:

print("Loaded model from disk")
from keras.models import model_from_json
json_file = open('imbalanced_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("imbalanced_data.h5")
model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                384       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)              

In [54]:
print("predicting....")
batch_size = 10000
sub['is_attributed'] = model.predict(test_df, batch_size=batch_size, verbose=2)
# del test_df; gc.collect()
print("writing....")
sub.to_csv('code2_sub.csv',index=False)

predicting....
writing....


In [59]:
sub.describe()


,click_id,is_attributed
count,1.879047e+07,1.879047e+07
mean,9.395234e+06,1.142960e-06
std,5.424341e+06,3.746760e-04
min,0.000000e+00,0.000000e+00
25%,4.697617e+06,0.000000e+00
50%,9.395234e+06,0.000000e+00
75%,1.409285e+07,0.000000e+00
max,1.879047e+07,1.916228e-01
